<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-25 09:52:00
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.41 C
-------------------
Today PnL: 49.91 K (0.36%)
Current PnL: -33.71 L (-21.07%)
CY Booked + Current PnL: -18.03 L (-11.27%)
-------------------
Total profit:  1.48 L
Total loss:  -35.19 L
-------------------
Total Booked + Current PnL: 8.95 L (6.73%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.14%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.00 C (71.29%)
Deployed:  1.33 C
Current:  1.41 C
CAGR/XIRR %: 2.89%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,0.74,6.76,10.22,17.67,16417.0,10176.0,160636.0,478.51,29.12,60.0,L-LC,11.25,185.0,0.62,1.16,28.95,XY25,ATH,MINING
50,MASFIN,0.87,4.44,16.40,21.57,16782.0,4350.0,102330.0,397.04,-10.12,60.0,H-SC,6.61,167.0,0.26,0.74,49.33,XR,ATH,FINANCE
77,TTKPRESTIG,-0.86,-30.33,43.66,0.09,30656.0,-30561.0,70216.0,770.00,63.07,35.0,M-SC,10.92,250.0,-1.00,0.51,1.17,OX40N,NTT,DURABLES
51,MEDANTA,0.43,-3.58,28.68,24.08,35107.0,-4541.0,122409.0,1486.00,-4.25,53.0,X-SC,8.92,87.0,-0.13,0.89,12.12,XY24,NTT,HEALTHCARE
36,ICICIGI,0.79,1.65,17.41,19.34,35412.0,3294.0,203403.0,2252.93,-14.44,53.0,X-MC,1.50,66.0,0.09,1.47,17.85,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BSOFT,2.76,-31.75,120.93,50.79,116087.0,-44658.0,95995.0,831.70,-12.25,39.0,H-SC,5.69,172.0,-0.38,0.69,11.77,XR,ATH,IT
42,INFY,2.41,-11.07,50.96,34.24,141792.0,-34646.0,278242.0,1972.00,-29.97,25.0,X-LC,7.11,3.0,-0.24,2.01,2.41,X40,NTT,IT
33,HCLTECH,2.40,-11.00,39.14,23.83,84272.0,-26614.0,215310.0,1908.19,-6.53,25.0,X-LC,4.64,8.0,-0.32,1.56,3.30,X40,ATH,IT
68,STARHEALTH,2.27,-9.82,63.30,47.26,160468.0,-27619.0,253504.0,761.00,42.33,53.0,H-SC,2.37,173.0,-0.17,1.83,35.82,XY24,NTT,INSURANCE
73,TCS,2.15,-24.65,64.00,23.58,208637.0,-106633.0,325996.0,4311.58,-34.78,29.0,X-LC,7.76,2.0,-0.51,2.36,2.15,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TRIDENT,-1.16,-31.54,87.72,28.51,55402.0,-29096.0,63158.0,48.00,-11.82,43.0,M-SC,8.70,223.0,-0.53,0.46,8.07,XR,NTT,TEXTILES
62,SATIN,-1.14,-15.22,79.08,51.83,185239.0,-42041.0,234243.0,274.00,-23.78,48.0,H-SC,3.66,159.0,-0.23,1.69,14.11,XY24,NTT,FINANCE
49,LAOPALA,-1.12,-38.25,125.24,39.09,101392.0,-50147.0,80958.0,464.00,77.16,57.0,H-SC,13.45,154.0,-0.49,0.59,8.26,AR,NTT,CERAMICS
19,COFFEEDAY,-1.12,-47.63,158.31,35.27,94134.0,-54087.0,59462.0,80.00,-59.76,43.0,L-SC,14.40,266.0,-0.57,0.43,45.54,XR,NTT,HOTELS
61,ROUTE,-1.06,-62.85,352.64,68.16,174324.0,-83632.0,49434.0,2215.45,-69.86,20.0,H-SC,30.02,141.0,-0.48,0.36,0.00,SR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.63,3.54,100.80,107.91,154095.0,5228.0,152872.0,1641.55,-6.65,59.0,M-SC,7.68,220.0,0.03,1.11,15.01,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.86,-30.33,43.66,0.09,30656.0,-30561.0,70216.0,770.00,63.07,35.0,M-SC,10.92,250.0,-1.00,0.51,1.17,OX40N,NTT,DURABLES
40,INDIGOPNTS,0.46,-32.32,47.79,0.02,56457.0,-56424.0,118135.0,1408.00,80.25,30.0,M-SC,11.30,222.0,-1.00,0.85,2.76,OX40N,NTT,PAINTS
66,SIS,-1.00,-33.15,81.38,21.26,60408.0,-36802.0,74230.0,528.00,1739.38,29.0,H-SC,4.94,168.0,-0.61,0.54,2.33,OX40N,NTT,MISC
48,KANSAINER,1.29,-33.18,69.82,13.47,125808.0,-89478.0,180189.0,340.00,-42.48,32.0,H-SC,11.93,162.0,-0.71,1.30,1.29,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,0.33,20.71,20.29,45.2,46961.0,39708.0,231450.0,1856.0,38.21,64.0,X-MC,7.83,79.0,0.85,1.67,28.91,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,0.74,6.76,10.22,17.67,16417.0,10176.0,160636.0,478.51,29.12,60.0,L-LC,11.25,185.0,0.62,1.16,28.95,XY25,ATH,MINING
26,FINCABLES,0.63,3.54,100.80,107.91,154095.0,5228.0,152872.0,1641.55,-6.65,59.0,M-SC,7.68,220.0,0.03,1.11,15.01,OX40N,ATH,CABLES
50,MASFIN,0.87,4.44,16.40,21.57,16782.0,4350.0,102330.0,397.04,-10.12,60.0,H-SC,6.61,167.0,0.26,0.74,49.33,XR,ATH,FINANCE
1,ABB,1.45,12.73,29.13,45.57,85908.0,33293.0,294912.0,7934.00,-27.57,70.0,H-MC,8.40,125.0,0.39,2.13,30.96,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,1.45,12.73,29.13,45.57,85908.0,33293.0,294912.0,7934.00,-27.57,70.0,H-MC,8.40,125.0,0.39,2.13,30.96,AR,NTT,ELECTRICAL
50,MASFIN,0.87,4.44,16.40,21.57,16782.0,4350.0,102330.0,397.04,-10.12,60.0,H-SC,6.61,167.0,0.26,0.74,49.33,XR,ATH,FINANCE
26,FINCABLES,0.63,3.54,100.80,107.91,154095.0,5228.0,152872.0,1641.55,-6.65,59.0,M-SC,7.68,220.0,0.03,1.11,15.01,OX40N,ATH,CABLES
39,INDIAMART,1.02,-3.94,115.19,106.72,136479.0,-4854.0,118482.0,4810.62,-56.64,53.0,H-SC,7.99,140.0,-0.04,0.86,18.52,AR,ATH,MISC
85,ZYDUSLIFE,0.24,-4.46,41.96,35.63,84015.0,-9348.0,200226.0,1286.17,-16.67,51.0,H-MC,5.05,120.0,-0.11,1.45,11.47,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,0.09,-37.80,152.72,57.19,309791.0,-123278.0,202849.0,485.00,-64.48,20.0,X-SC,18.25,81.0,-0.40,1.47,0.09,XY24,NTT,FMCG
33,HCLTECH,2.40,-11.00,39.14,23.83,84272.0,-26614.0,215310.0,1908.19,-6.53,25.0,X-LC,4.64,8.0,-0.32,1.56,3.30,X40,ATH,IT
42,INFY,2.41,-11.07,50.96,34.24,141792.0,-34646.0,278242.0,1972.00,-29.97,25.0,X-LC,7.11,3.0,-0.24,2.01,2.41,X40,NTT,IT
12,BATAINDIA,-0.26,-48.59,163.00,35.20,107820.0,-62523.0,66147.0,2096.00,-13.21,29.0,X-SC,22.01,91.0,-0.58,0.48,0.00,X40,NTT,FOOTWEAR
73,TCS,2.15,-24.65,64.00,23.58,208637.0,-106633.0,325996.0,4311.58,-34.78,29.0,X-LC,7.76,2.0,-0.51,2.36,2.15,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,-0.32,4.09,16.02,20.77,39045.0,9580.0,243724.0,1644.00,-7.70,58.0,X-MC,0.85,64.0,0.25,1.76,11.80,X40N,NTT,BREWERIES
21,DABUR,0.67,1.96,40.94,43.70,103549.0,4860.0,252928.0,735.00,-7.46,57.0,X-MC,1.45,70.0,0.05,1.83,17.30,XY24,BTT,FMCG
36,ICICIGI,0.79,1.65,17.41,19.34,35412.0,3294.0,203403.0,2252.93,-14.44,53.0,X-MC,1.50,66.0,0.09,1.47,17.85,X40,ATH,INSURANCE
59,RELIANCE,0.25,-8.06,36.28,25.30,67033.0,-16196.0,184767.0,1952.00,-10.42,48.0,X-LC,2.68,21.0,-0.24,1.34,23.76,XY25,BTT,REFINERIES
34,HINDUNILVR,0.56,-7.73,23.20,13.67,55026.0,-19883.0,237180.0,2922.00,-31.53,52.0,X-LC,2.69,9.0,-0.36,1.72,11.97,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-0.26,-48.59,163.00,35.20,107820.0,-62523.0,66147.0,2096.00,-13.21,29.0,X-SC,22.01,91.0,-0.58,0.48,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,-0.41,-25.51,108.72,55.47,197028.0,-62061.0,181225.0,181.50,-36.04,31.0,X-MC,17.05,60.0,-0.31,1.31,0.00,X40N,ATH,FINANCE
58,RELAXO,-0.13,-54.35,233.62,52.31,154820.0,-78890.0,66270.0,1176.00,-51.41,37.0,X-SC,14.33,92.0,-0.51,0.48,0.00,X40N,NTT,FOOTWEAR
53,PGHH,-0.28,-14.12,53.71,32.01,98491.0,-30144.0,183376.0,17616.87,-41.89,31.0,X-MC,9.65,57.0,-0.31,1.33,0.00,X40,ATH,FMCG
8,AWL,0.09,-37.80,152.72,57.19,309791.0,-123278.0,202849.0,485.00,-64.48,20.0,X-SC,18.25,81.0,-0.40,1.47,0.09,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-0.41,-32.92,112.96,42.84,49254.0,-21403.0,43603.0,424.00,-55.92,42.0,X-SC,17.62,80.0,-0.43,0.32,4.88,XY24,NTT,MISC
12,BATAINDIA,-0.26,-48.59,163.00,35.20,107820.0,-62523.0,66147.0,2096.00,-13.21,29.0,X-SC,22.01,91.0,-0.58,0.48,0.00,X40,NTT,FOOTWEAR
58,RELAXO,-0.13,-54.35,233.62,52.31,154820.0,-78890.0,66270.0,1176.00,-51.41,37.0,X-SC,14.33,92.0,-0.51,0.48,0.00,X40N,NTT,FOOTWEAR
51,MEDANTA,0.43,-3.58,28.68,24.08,35107.0,-4541.0,122409.0,1486.00,-4.25,53.0,X-SC,8.92,87.0,-0.13,0.89,12.12,XY24,NTT,HEALTHCARE
35,HONAUT,0.35,-22.10,83.98,43.32,106554.0,-35992.0,126880.0,58357.33,-31.63,45.0,X-SC,11.33,90.0,-0.34,0.92,3.02,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.12,-14.90,39.89,19.06,115481.0,-50668.0,289498.0,452.00,-53.50,47.0,X-LC,17.44,1.0,-0.44,2.09,4.82,X40,NTT,FMCG
73,TCS,2.15,-24.65,64.00,23.58,208637.0,-106633.0,325996.0,4311.58,-34.78,29.0,X-LC,7.76,2.0,-0.51,2.36,2.15,X40,ATH,IT
42,INFY,2.41,-11.07,50.96,34.24,141792.0,-34646.0,278242.0,1972.00,-29.97,25.0,X-LC,7.11,3.0,-0.24,2.01,2.41,X40,NTT,IT
75,TMPV,-0.03,-16.12,58.96,33.33,161122.0,-52526.0,273274.0,600.00,-82.53,57.0,X-LC,3.33,4.0,-0.33,1.98,11.72,XY24,NTT,AUTO
33,HCLTECH,2.40,-11.00,39.14,23.83,84272.0,-26614.0,215310.0,1908.19,-6.53,25.0,X-LC,4.64,8.0,-0.32,1.56,3.30,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,0.62,-30.81,92.06,32.89,49178.0,-23786.0,53420.0,1800.00,-351.40,59.0,L-MC,11.59,258.0,-0.48,0.39,47.12,XR,NTT,BANKS
67,SONACOMS,0.88,-7.60,50.54,39.09,47239.0,-7691.0,93468.0,804.02,-26.82,62.0,M-MC,10.01,193.0,-0.16,0.68,32.23,AR,ATH,AUTO
50,MASFIN,0.87,4.44,16.40,21.57,16782.0,4350.0,102330.0,397.04,-10.12,60.0,H-SC,6.61,167.0,0.26,0.74,49.33,XR,ATH,FINANCE
18,COALINDIA,0.74,6.76,10.22,17.67,16417.0,10176.0,160636.0,478.51,29.12,60.0,L-LC,11.25,185.0,0.62,1.16,28.95,XY25,ATH,MINING
65,SIEMENS,0.22,-9.38,38.50,25.51,64928.0,-17450.0,168645.0,4671.50,41.70,66.0,H-LC,2.98,51.0,-0.27,1.22,24.28,AR,ATH,ELECTRICAL


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,-0.68,-51.99,203.45,45.70,172719.0,-91920.0,84895.0,26.4,-11.99,63.0,M-SC,18.60,204.0,-0.53,0.61,39.87,XY24,NTT,TRAVEL
1,ABB,1.45,12.73,29.13,45.57,85908.0,33293.0,294912.0,7934.0,-27.57,70.0,H-MC,8.40,125.0,0.39,2.13,30.96,AR,NTT,ELECTRICAL
7,ATULAUTO,1.25,-17.33,76.09,45.57,123998.0,-34170.0,162962.0,844.0,3527.03,54.0,M-SC,5.13,248.0,-0.28,1.18,23.32,XY24,NTT,AUTO
11,BANDHANBNK,-0.80,-13.84,123.34,92.42,296009.0,-38565.0,239994.0,400.0,106.18,73.0,H-SC,7.01,174.0,-0.13,1.74,33.55,XY24,NTT,BANKS
83,WHIRLPOOL,1.94,-26.13,146.27,81.92,124038.0,-29997.0,84801.0,2270.0,-56.73,59.0,M-SC,26.15,210.0,-0.24,0.61,19.12,XR,NTT,DURABLES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.70
1,25,45.63
2,50,77.51


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.01
MC    30.24
LC    24.78
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.96
X40      23.86
X40N     14.80
AR        9.11
XY25      8.92
XR        8.45
OX40N     7.11
SR        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.73
H-SC    22.15
X-LC    21.38
X-SC    11.51
M-SC    10.62
H-MC     5.03
M-MC     2.09
H-LC     1.22
L-LC     1.16
M-LC     1.02
L-SC     0.73
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.69,-13.81,52.30
FINANCE,13.09,-15.38,60.78
IT,11.25,-40.85,111.44
MISC,6.88,-36.71,92.63
ELECTRICAL,6.39,-6.18,44.92
PAINTS,5.04,-32.30,51.96
INSURANCE,4.87,-0.70,35.16
PHARMA,4.11,-3.35,35.88
AUTO,3.84,-17.82,62.74


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3400394.0,21
X40,1422400.0,15
XR,1322382.0,12
AR,1311890.0,10
X40N,1076486.0,10
OX40N,809524.0,10
XY25,383721.0,6
SR,305897.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3730384.0,24
M-SC,1447877.0,14
X-MC,1368440.0,15
X-LC,1318881.0,12
X-SC,1215727.0,10
H-MC,383413.0,3
L-SC,189761.0,2
M-LC,150342.0,1
M-MC,97346.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1259690.0      6
           AR         954662.0      5
           XR         849288.0      7
M-SC       XY24       840249.0      6
X-LC       X40        695719.0      6
X-SC       XY24       622484.0      4
X-MC       X40        543714.0      7
           X40N       437909.0      4
X-SC       X40N       410276.0      4
H-SC       OX40N      360847.0      4
M-SC       OX40N      353050.0      5
H-SC       SR         305897.0      2
X-LC       XY24       272802.0      2
           X40N       228301.0      2
H-MC       XY24       213490.0      1
X-MC       XY25       195138.0      2
           XY24       191679.0      2
X-SC       X40        182967.0      2
M-SC       XR         179440.0      2
H-MC       AR         169923.0      2
M-LC       XR         150342.0      1
X-LC       XY25       122059.0      2
L-SC       OX40N       95627.0      1
           XR          94134.0      1
M-SC       AR          75138.0      1
H-LC       AR          64928.0      1
M-MC       XY25        50107.0      1
L-MC       XR          49178.0      1
M-MC       AR          47239.0      1
L-LC       XY25        16417.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
